# NMF Clustering in GenePattern Notebook

Non-negative matrix factorization (NMF) finds a small number of metagenes, each defined as a positive linear combination of the genes in the expression data. It then groups samples into clusters based on the gene expression pattern of these metagenes.

## Before you begin

* Sign in to GenePattern by entering your username and password into the form below. If you are seeing a block of code instead of the login form, go to the menu above and select Cell > Run All.
* Gene expression data must be in a [GCT or RES file](https://genepattern.broadinstitute.org/gp/pages/protocols/GctResFiles.html).
    * Example file: [all_aml_test.gct](ftp://ftp.broadinstitute.org/pub/genepattern/datasets/all_aml/all_aml_test.gct).
* The gene expression data must contain only positive values. If your data contains negative values, see the [NMFConsensus documentation](http://genepatternbeta.broadinstitute.org/gp/getTaskDoc.jsp?name=NMFConsensus) for instructions.
* Learn more by reading about [file formats](http://www.broadinstitute.org/cancer/software/genepattern/file-formats-guide#GCT).


In [1]:
# !AUTOEXEC

%reload_ext genepattern

# Don't have the GenePattern Notebook? It can be installed from PIP: 
# pip install genepattern-notebook 
import gp

# The following widgets are components of the GenePattern Notebook extension.
try:
    from genepattern import GPAuthWidget, GPJobWidget, GPTaskWidget
except:
    def GPAuthWidget(input):
        print("GP Widget Library not installed. Please visit http://genepattern.org")
    def GPJobWidget(input):
        print("GP Widget Library not installed. Please visit http://genepattern.org")
    def GPTaskWidget(input):
        print("GP Widget Library not installed. Please visit http://genepattern.org")

# The gpserver object holds your authentication credentials and is used to
# make calls to the GenePattern server through the GenePattern Python library.
# Your actual username and password have been removed from the code shown
# below for security reasons.
gpserver = gp.GPServer("https://genepattern.broadinstitute.org/gp", "", "")

# Return the authentication widget to view it
GPAuthWidget(gpserver)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


## Step 1: PreprocessDataset

Preprocess gene expression data to remove platform noise and genes that have little variation. Although researchers generally preprocess data before clustering if doing so removes relevant biological information, skip this step. 

### Considerations

* PreprocessDataset can preprocess the data in one or more ways (in this order):
    1. Set threshold and ceiling values. Any value lower/higher than the threshold/ceiling value is reset to the threshold/ceiling value.
    2. Convert each expression value to the log base 2 of the value.
    3. Remove genes (rows) if a given number of its sample values are less than a given threshold.
    4. Remove genes (rows) that do not have a minimum fold change or expression variation.
    5. Discretize or normalize the data.
* When using ratios to compare gene expression between samples, convert values to log base 2 of the value to bring up- and down-regulated genes to the same scale. For example, ratios of 2 and .5 indicating two-fold changes for up- and down-regulated expression, respectively, are converted to +1 and -1. 
* If you did not generate the expression data, check whether preprocessing steps have already been taken before running the PreprocessDataset module. 
* Learn more by reading about the [PreprocessDataset](https://genepattern.broadinstitute.org/gp/getTaskDoc.jsp?name=PreprocessDataset) module.

In [2]:
# !AUTOEXEC

preprocessdataset_task = gp.GPTask(gpserver, 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00020:5.1')
preprocessdataset_job_spec = preprocessdataset_task.make_job_spec()
preprocessdataset_job_spec.set_parameter("input.filename", "ftp://ftp.broadinstitute.org/pub/genepattern/datasets/all_aml/all_aml_test.gct")
preprocessdataset_job_spec.set_parameter("threshold.and.filter", "1")
preprocessdataset_job_spec.set_parameter("floor", "20")
preprocessdataset_job_spec.set_parameter("ceiling", "20000")
preprocessdataset_job_spec.set_parameter("min.fold.change", "3")
preprocessdataset_job_spec.set_parameter("min.delta", "100")
preprocessdataset_job_spec.set_parameter("num.outliers.to.exclude", "0")
preprocessdataset_job_spec.set_parameter("row.normalization", "0")
preprocessdataset_job_spec.set_parameter("row.sampling.rate", "1")
preprocessdataset_job_spec.set_parameter("threshold.for.removing.rows", "")
preprocessdataset_job_spec.set_parameter("number.of.columns.above.threshold", "")
preprocessdataset_job_spec.set_parameter("log2.transform", "0")
preprocessdataset_job_spec.set_parameter("output.file.format", "3")
preprocessdataset_job_spec.set_parameter("output.file", "<input.filename_basename>.preprocessed")
GPTaskWidget(preprocessdataset_task)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


## Step 2: NMFConsensus

NMFConsensus uses the basic principle of dimensionality reduction via non-negative matrix factorization (NMF) to find a small number of metagenes, each defined as a positive linear combination of the genes in the expression data. It then groups samples into clusters based on the gene expression pattern of the samples as positive linear combinations of these metagenes. NMFConsensus repeatedly runs the clustering algorithm against perturbations of the gene expression data and creates a consensus matrix to assesses the stability of the resulting clusters.

**3-4 hours:** Running this example on the GenePattern public server takes several hours. The results are provided here for your convenience: [NMFConsensus_Results.zip](ftp://ftp.broadinstitute.org/pub/genepattern/datasets/protocols/NMFConsensus_Results.zip). 

### Considerations

* Non-negative matrix factorization (NMF) requires positive gene expression values. To run NMF on data that contains negative values (Kim & Tidor, 2003): 
    1. Create one dataset with all negative numbers zeroed.
    2. Create another dataset with all positive numbers zeroed and the signs of all negative numbers removed. 
    3. Merge the two (eg. by concatenation), resulting in a dataset twice as large as the original, but with positive values only and zeros, hence appropriate for NMF. 
* To do this in MATLAB, execute the following statement:
        anew=[max(a,0);-min(a,0)];
        where a is the original data.
* NMFConsensus groups samples into k classes based on k metagenes. Use the k initial and k final parameters to specify the desired number of classes (by default, 2-5). NMFConsensus builds a separate consensus matrix for each set of classes.
* This module executes an R version of NMFConsensus, which is slow and is intended for exploratory use. A faster MATLAB version of NMFConsensus is available on the [Broad Institute web site](http://www.broadinstitute.org/cgi-bin/cancer/publications/pub_paper.cgi?mode=view&paper_id=89). 
* Learn more by reading about the [NMFConsensus](https://genepattern.broadinstitute.org/gp/getTaskDoc.jsp?name=NMFConsensus) module.

In [3]:
# !AUTOEXEC

nmfconsensus_task = gp.GPTask(gpserver, 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00057:5')
nmfconsensus_job_spec = nmfconsensus_task.make_job_spec()
nmfconsensus_job_spec.set_parameter("dataset.filename", "ftp://ftp.broadinstitute.org/pub/genepattern/datasets/protocols/all_aml_test.preprocessed.gct")
nmfconsensus_job_spec.set_parameter("k.initial", "2")
nmfconsensus_job_spec.set_parameter("k.final", "5")
nmfconsensus_job_spec.set_parameter("num.clusterings", "20")
nmfconsensus_job_spec.set_parameter("max.num.iterations", "2000")
nmfconsensus_job_spec.set_parameter("error.function", "divergence")
nmfconsensus_job_spec.set_parameter("random.seed", "123456789")
nmfconsensus_job_spec.set_parameter("output.file.prefix", "<dataset.filename_basename>")
nmfconsensus_job_spec.set_parameter("stop.convergence", "40")
nmfconsensus_job_spec.set_parameter("stop.frequency", "10")
GPTaskWidget(nmfconsensus_task)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


## Step 3: View Results

Plots of the results are written to .pdf files. Cluster membership results are written to GCT files. View the result files by clicking on them. 

* For an overview of the results, click *.consensus.all.k.plot.pdf. A consensus matrix where all values are dark blue (0) or dark red (1) corresponds to perfect consensus. For example: [all_aml_test.preprocessed.consensus.all.k.plot.pdf](ftp://ftp.broadinstitute.org/pub/genepattern/datasets/protocols/all_aml_test.preprocessed.consensus.all.k.plot.pdf). 
* For a complete listing of cluster membership, click *.membership.gct.

### Considerations

* For more about the plots and their interpretation, see Brunet et al., 2004.
* Learn more by reading about the [NMFConsensus](https://genepattern.broadinstitute.org/gp/getTaskDoc.jsp?name=NMFConsensus) module.